# Caso 3 - Jimena Chacón 

In [237]:
import pandas as pd
import numpy as np
import QuantLib as ql
import re

### a) Enfoque paramétrico utilizando aproximación Delta-Normal

In [238]:
#Cálculo para el bono 1

p1= 99.849511
yieldrate1= 0.03875
periodicidad1 = 2

In [239]:
fecha_emision = pd.to_datetime("2023-03-31")
fecha_vence = pd.to_datetime("2025-03-31")
fecha_calculo = pd.to_datetime("2023-03-31")

In [240]:
fecha_calc_ql = ql.Date(fecha_calculo.day,fecha_calculo.month,fecha_calculo.year)

ql.Settings.instance().evaluationDate = fecha_calc_ql

In [241]:
qlCalendar = ql.BespokeCalendar("United States")
qlCalendar.addWeekend(ql.Sunday)
qlCalendar.addWeekend(ql.Saturday)

In [242]:
fecha_vence_ql = ql.Date(fecha_vence.day,fecha_vence.month,fecha_vence.year)
fecha_emision_ql = ql.Date(fecha_emision.day,fecha_emision.month,fecha_emision.year)

In [243]:
qlTenor = ql.Period(periodicidad1)
qlConvencion = ql.Following
dateGeneration = ql.DateGeneration.Backward
monthEnd = False

In [244]:
cronograma = ql.Schedule(fecha_emision_ql,fecha_vence_ql,qlTenor, qlCalendar,qlConvencion,qlConvencion,dateGeneration,monthEnd)

pd.DataFrame({"date":list(cronograma)})

,date
0,"March 31st, 2023"
1,"October 2nd, 2023"
2,"April 1st, 2024"
3,"September 30th, 2024"
4,"March 31st, 2025"


In [245]:
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)
compound_type = ql.Compounded
frequency = ql.Semiannual
settlementDays = 0

In [246]:
interest_rate = ql.InterestRate(yieldrate1,dayCount,compound_type,frequency)
print(interest_rate)

3.875000 % 30/360 (Bond Basis) Semiannual compounding


In [247]:
bono1 = ql.FixedRateBond(settlementDays,100,cronograma,[yieldrate1],dayCount)

In [248]:
cf = bono1.cashflows()
fechas = [item.date() for item in cf]
montos = [item.amount() for item in cf]
print(pd.DataFrame({"Fechas":fechas, "Montos": montos}))

                 Fechas      Montos
0     October 2nd, 2023    1.959028
1       April 1st, 2024    1.926736
2  September 30th, 2024    1.926736
3      March 31st, 2025    1.937500
4      March 31st, 2025  100.000000


In [249]:
rend = bono1.bondYield(p1,dayCount,compound_type,frequency)

In [250]:
interes_rend = ql.InterestRate(rend,dayCount,compound_type,frequency)

In [251]:
duracionMod1 = ql.BondFunctions.duration(bono1,interes_rend,ql.Duration.Modified)

In [252]:
curvas = pd.read_excel("CurvaActualizadaCaso3 (1).xlsx")
curvas.set_index('Date', inplace=True)
curva_actual = curvas.iloc[-1,:]

In [253]:
tasa_actual = curva_actual.values
print(tasa_actual)
plazos = curva_actual.index.values
plazos_numeros = [int(re.search(r'\d+', plazo).group()) for plazo in plazos]
plazos_numeros_array = np.array(plazos_numeros)
plazos_numeros_array

[0.05343767 0.05259337 0.05097032 0.04881758 0.04535153 0.04079181
 0.03817697 0.03579613 0.03510458 0.03521748]


array([   1,   30,   90,  180,  360,  720, 1080, 1800, 2520, 3600])

In [254]:
fecha_calc_ql

Date(31,3,2023)

In [255]:
curva_ql0 = ql.ZeroCurve(fecha_calc_ql+plazos_numeros_array,tasa_actual,ql.Thirty360(ql.Thirty360.BondBasis))
curva_ql = ql.YieldTermStructureHandle(curva_ql0)
estructura_plazos = ql.ZeroSpreadedTermStructure(curva_ql,ql.QuoteHandle(ql.SimpleQuote(0)),ql.Compounded,ql.Semiannual,dayCount)

In [256]:
pricing_engine = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(estructura_plazos))

In [257]:
bono1.setPricingEngine(pricing_engine)
print(bono1.NPV())

99.52623809960488


In [258]:
def Precio1(spread):
    estructura_plazos = ql.ZeroSpreadedTermStructure(curva_ql,ql.QuoteHandle(ql.SimpleQuote(spread)),ql.Compounded,ql.Semiannual,dayCount)
    pricing_engine = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(estructura_plazos))

    bono1.setPricingEngine(pricing_engine)
    return bono1.NPV()

In [259]:
from scipy.optimize import bisect

In [260]:
def ErrorPrecio1(spread):
    return p1-Precio1(spread)

In [261]:
spread_estimado1 = bisect(ErrorPrecio1,-1,1)
spread_estimado1

-0.0017021180010488024

In [262]:
Precio1(spread_estimado1)

99.8495110002618

In [263]:
#Cálculo para el bono 2

p2= 99.993866
yieldrate2= 0.03625

In [264]:
fecha_emision2 = pd.to_datetime("2023-03-31")
fecha_vence2 = pd.to_datetime("2030-03-31")
fecha_calculo2 = pd.to_datetime("2023-03-31")

In [265]:
fecha_calc_ql2 = ql.Date(fecha_calculo2.day,fecha_calculo2.month,fecha_calculo2.year)

ql.Settings.instance().evaluationDate = fecha_calc_ql2

In [266]:
fecha_vence_ql2 = ql.Date(fecha_vence2.day,fecha_vence2.month,fecha_vence2.year)
fecha_emision_ql2 = ql.Date(fecha_emision2.day,fecha_emision2.month,fecha_emision2.year)

In [267]:
cronograma2 = ql.Schedule(fecha_emision_ql2,fecha_vence_ql2,qlTenor, qlCalendar,qlConvencion,qlConvencion,dateGeneration,monthEnd)

pd.DataFrame({"date":list(cronograma2)})

,date
0,"March 31st, 2023"
1,"October 2nd, 2023"
2,"April 1st, 2024"
3,"September 30th, 2024"
4,"March 31st, 2025"
5,"September 30th, 2025"
6,"March 31st, 2026"
7,"September 30th, 2026"
8,"March 31st, 2027"
9,"September 30th, 2027"


In [268]:
interest_rate2 = ql.InterestRate(yieldrate2,dayCount,compound_type,frequency)
print(interest_rate2)

3.625000 % 30/360 (Bond Basis) Semiannual compounding


In [269]:
bono2 = ql.FixedRateBond(settlementDays,100,cronograma2,[yieldrate2],dayCount)

In [270]:
cf = bono2.cashflows()
fechas = [item.date() for item in cf]
montos = [item.amount() for item in cf]
print(pd.DataFrame({"Fechas":fechas, "Montos": montos}))

                  Fechas      Montos
0      October 2nd, 2023    1.832639
1        April 1st, 2024    1.802431
2   September 30th, 2024    1.802431
3       March 31st, 2025    1.812500
4   September 30th, 2025    1.812500
5       March 31st, 2026    1.812500
6   September 30th, 2026    1.812500
7       March 31st, 2027    1.812500
8   September 30th, 2027    1.812500
9       March 31st, 2028    1.812500
10     October 2nd, 2028    1.832639
11       April 2nd, 2029    1.812500
12     October 1st, 2029    1.802431
13       April 1st, 2030    1.812500
14       April 1st, 2030  100.000000


In [271]:
rend2 = bono2.bondYield(p2,dayCount,compound_type,frequency)

In [272]:
interes_rend2 = ql.InterestRate(rend2,dayCount,compound_type,frequency)

In [273]:
duracionMod2 = ql.BondFunctions.duration(bono2,interes_rend2,ql.Duration.Modified)

In [274]:
fecha_calc_ql2

Date(31,3,2023)

In [275]:
bono2.setPricingEngine(pricing_engine)
print(bono2.NPV())

100.37343491942642


In [276]:
def Precio2(spread):
    estructura_plazos = ql.ZeroSpreadedTermStructure(curva_ql,ql.QuoteHandle(ql.SimpleQuote(spread)),ql.Compounded,ql.Semiannual,dayCount)
    pricing_engine = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(estructura_plazos))

    bono2.setPricingEngine(pricing_engine)
    return bono2.NPV()

In [277]:
def ErrorPrecio2(spread):
    return p2-Precio2(spread)

In [278]:
spread_estimado2 = bisect(ErrorPrecio2,-1,1)
spread_estimado2

0.0006169575153762707

In [279]:
Precio2(spread_estimado2)

99.99386600013406

In [280]:
from scipy.stats import norm

In [281]:
nivel_conf = 0.999
zq = norm.ppf(nivel_conf)
zq

3.090232306167813

In [282]:
cambio_tasas = curvas.diff().iloc[1:]

In [283]:
sigma_plazo = cambio_tasas.std()
vol = sigma_plazo.iloc[9]
vol

0.0006080116336959645

In [284]:
N1 = 1000000
N2 = 1500000
M1= N1*p1
M2= N2*p2
valor_cartera = M1+M2

In [285]:
VaR = M1*duracionMod1*zq*vol+M2*duracionMod2*zq*vol
print("El riesgo del portafolio por aproximación Delta-Normal es:", VaR)

El riesgo del portafolio por aproximación Delta-Normal es: 2086771.9194314259


### b) Enfoque por componentes principales

In [286]:
from sklearn import decomposition

In [287]:
pca = decomposition.PCA(n_components=3,whiten=False)

In [288]:
cambios_tasas = np.diff(curvas.values,axis=0)
cambios_tasas

array([[-1.65749015e-04, -1.46329537e-04, -1.08865984e-04, ...,
         1.72131446e-04,  1.45750538e-04,  1.07040829e-04],
       [-7.44306354e-05, -6.41844571e-05, -4.45633100e-05, ...,
         8.21708170e-05,  7.37978210e-05,  8.18675202e-05],
       [ 2.23922316e-04,  1.69945274e-04,  6.52711556e-05, ...,
        -7.93449980e-04, -7.25514945e-04, -6.09650102e-04],
       ...,
       [ 4.47665163e-05,  7.83767079e-05,  1.38323416e-04, ...,
         2.82785082e-04,  2.48856720e-04,  2.11322824e-04],
       [ 9.41414098e-04,  8.91434923e-04,  7.94245291e-04, ...,
        -1.45335103e-04, -1.89366493e-04, -2.21481527e-04],
       [ 1.30009353e-03,  1.13667587e-03,  8.35302629e-04, ...,
        -6.49598072e-04, -6.18249935e-04, -6.37541540e-04]])

In [289]:
x_r = pca.fit_transform(cambios_tasas)

In [290]:
def PriceTermStructShift(spread,shift,bono):
    ql_curvacero_shift = ql.ZeroCurve(fecha_calc_ql+plazos_numeros_array,tasa_actual+shift,ql.Thirty360(ql.Thirty360.BondBasis))
    ql_curva_shift = ql.YieldTermStructureHandle(ql_curvacero_shift)
    term_structure_spread = ql.ZeroSpreadedTermStructure(ql_curva_shift,ql.QuoteHandle(ql.SimpleQuote(spread)),ql.Compounded,ql.Semiannual,dayCount)
    pricing_engine_shift = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(term_structure_spread))
    bono.setPricingEngine(pricing_engine_shift)
    return bono.NPV()

In [291]:
comp = pca.components_

In [292]:
dur_pca = np.zeros(3)
delta = .0001
for icomp in range(0,3):
    nuevo_precio = PriceTermStructShift(spread_estimado1,delta*comp[icomp],bono1)
    dur_pca[icomp] = (1/p1)*(p1-nuevo_precio)/(delta)
print(dur_pca)

[ 0.00388177 -0.79096279  0.0326587 ]


In [293]:
dur_pca2 = np.zeros(3)
for icomp in range(0,3):
    nuevo_precio2 = PriceTermStructShift(spread_estimado2,delta*comp[icomp],bono2)
    dur_pca2[icomp] = (1/p2)*(p2-nuevo_precio2)/(delta)
print(dur_pca2)

[ 0.03464572 -2.66356874  0.22351288]


In [294]:
from numpy.linalg import norm

In [295]:
covar = np.cov(np.transpose(x_r)) #valores propios de los componentes principales en la diagonal
covar

array([[1.24052678e-04, 1.87670999e-22, 4.28962284e-22],
       [1.87670999e-22, 1.84952635e-06, 5.46256658e-22],
       [4.28962284e-22, 5.46256658e-22, 1.04828946e-06]])

In [296]:
sigma = np.sqrt(np.dot(np.dot(np.transpose(dur_pca),covar),dur_pca))
print(sigma)

0.001077075645289994


In [297]:
sigma2 = np.sqrt(np.dot(np.dot(np.transpose(dur_pca2),covar),dur_pca2))
print(sigma)

0.001077075645289994


In [298]:
VaR = M1*sigma*zq+M2*sigma2*zq
print("El riesgo del portafolio por análisis de componentes principales es:", VaR)

El riesgo del portafolio por análisis de componentes principales es: 2024165.635233649


### c) Enfoque por simulación histórica

In [299]:
n = len(cambios_tasas)
precios_historicos1 = np.zeros(n)
for i in range(0,n):
    precios_historicos1[i] = PriceTermStructShift(spread_estimado1, cambios_tasas[i,:],bono1)

precios_historicos1

array([99.82662189, 99.83660687, 99.96510786, ..., 99.78351626,
       99.81850077, 99.95150933])

In [300]:
precios_historicos2 = np.zeros(n)
for i in range(0,n):
    precios_historicos2[i] = PriceTermStructShift(spread_estimado2, cambios_tasas[i,:],bono2)

precios_historicos2

array([ 99.90330555,  99.94747637, 100.44553635, ...,  99.83632727,
       100.10393461, 100.37984557])

In [301]:
valor_cartera_historico = N1*precios_historicos1+N2*precios_historicos2

In [302]:
VaRHist = (valor_cartera-np.quantile(valor_cartera_historico,1-nivel_conf))
VaRHist
print("El riesgo del portafolio por aproximación Delta-Normal es:", VaRHist)

El riesgo del portafolio por aproximación Delta-Normal es: 2490484.6527523696
